In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

In [2]:
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  x1,x2=x
  return np.sqrt(4+x1**2) + np.sqrt(4+x2**2)

def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  x1,x2=x
  return np.array([x1/np.sqrt(x1**2+4), x2/np.sqrt(x2**2+4)])  

def evalh(x): 
  assert type(x) is np.ndarray 
  assert len(x) == 2
  x1,x2 = x
  return np.array([[4/np.power(x1**2+4,1.5),0],[0,4/np.power(x2**2+4,1.5)]])

In [3]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #compute and return D_k
  hes = evalh(x)
  vals = np.linalg.eigvals(hes)
  return np.array([[1/vals[0],0],[0,1/vals[1]]])



def compute_steplength_backtracking_scaled_direction(x, gradf, d, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(d) is np.ndarray and len(d) == 2 
  assert alpha_start>=0.
  assert type(rho) is float and rho>=0
  assert type(gamma) is float and gamma>=0

  alp = alpha_start
  x = np.array(x,dtype='float64')
  while evalf(x - np.multiply(alp,d@gradf)) > evalf(x) - gamma*alp*np.matmul(gradf,d@gradf.T):
    alp *= rho
  return alp


def find_minimizer_gdscaling(start_x, tol,alpha,rho,gamma):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  k = 0
  
  while np.linalg.norm(evalg(x)) > tol:
    D = compute_D_k(x)
    step = compute_steplength_backtracking_scaled_direction(x,evalg(x),D,alpha,rho,gamma)
    x = x - np.multiply(step,np.matmul(D,evalg(x)))
   
    k+= 1
    if k%10000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
  return x,evalf(x),k 

def find_minimizer_gd(start_x, tol,alpha, rho, gamma):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  
  x= start_x
  k = 0
 
  while np.linalg.norm(evalg(x)) > tol: 
    step = compute_steplength_backtracking(x,evalg(x),alpha,rho,gamma)
    x =x- np.multiply(step,evalg(x))
    k += 1
    if k%10000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

  return x, evalf(x),k  

In [4]:
def find_minimizer_newton(start_x, tol,step=1,backtrack=False):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  k = 0
  
  while np.linalg.norm(evalg(x)) > tol :
    D = np.linalg.inv(evalh(x))
  
    if backtrack:
      alp = compute_steplength_backtracking_scaled_direction(x,evalg(x),D,step,0.5,0.5)
    else:
      alp=step
 
    x =x - np.multiply(alp,np.matmul(D,evalg(x)))
    k+= 1
    if k%10000==0:
        print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')

    
  print(f'Iteration:{k}, Minimizer:{x}, min value:{evalf(x)}, gradient norm:{np.linalg.norm(evalg(x))}')
  return x,evalf(x),k

def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  
  #Complete the code 
  alp = alpha_start
  while evalf(x - np.multiply(alp,gradf)) > evalf(x) - gamma*alp*np.matmul(gradf,gradf.T):
    alp *= rho

  return alp    

## 2)

In [5]:
X = np.array([2,2])
tol = 1e-9

x1,fx1,k1 = find_minimizer_newton(X,tol,0.4)

Iteration:40, Minimizer:[8.55212621e-10 8.55212621e-10], min value:4.0, gradient norm:6.047266434127795e-10


In [6]:
x2, fx2, k2 = find_minimizer_newton(X,tol,backtrack=True)

Iteration:1, Minimizer:[0. 0.], min value:4.0, gradient norm:0.0


In [7]:
print(f'Newtons method with constant step length 0.4 : Minimizer {x1}, Min function value :{fx1} , Iterations:{k1}.\nNewtons method with backtracking line search: Minimizer {x2}, Min function value :{fx2} , Iterations:{k2}.')

Newtons method with constant step length 0.4 : Minimizer [8.55212621e-10 8.55212621e-10], Min function value :4.0 , Iterations:40.
Newtons method with backtracking line search: Minimizer [0. 0.], Min function value :4.0 , Iterations:1.


Actual minimizer: [0,0]\
Min function value: 4

Newtons method with step length = 1 leads to the minimizer oscillating between (2,2) and (-2,-2).hence we dont get a solution.

If we reduce the step size to 0.4 (since we shud choose a value less than half of 1), it converges in 40 iterations.Error in minimizer is of the order 1e-10

With backtracking, it takes just 1 iteration, since the optimal step size happens to be 0.5, which is a multiple of rho.\
(For different values of alpha and rho, it would have taken more iterations)
It converges to exact solution, so there is no error.

## 3)

In [8]:
x3, fx3, k3 = find_minimizer_gdscaling(X,tol,1,0.5,0.5)


Iteration:1, Minimizer:[0. 0.], min value:4.0, gradient norm:0.0


In [9]:
x4, fx4, k4 = find_minimizer_gd(X,tol,1,0.5,0.5)


Iteration:32, Minimizer:[7.62525638e-10 7.62525638e-10], min value:4.0, gradient norm:5.391870494256027e-10


In [10]:
print(f'Gradient descent with scaling : Minimizer {x3}, Min function value :{fx3} , Iterations:{k3}.\nGradient descent without scaling: Minimizer {x4}, Min function value :{fx4} , Iterations:{k4}.')

Gradient descent with scaling : Minimizer [0. 0.], Min function value :4.0 , Iterations:1.
Gradient descent without scaling: Minimizer [7.62525638e-10 7.62525638e-10], Min function value :4.0 , Iterations:32.


Since the hessian matrix is diagonal, it happens to coincide with our chosen scaling matrix. Hence, we get same solution for newtons method and scaled gradient descent (exact solution(0,0).\
GD with scaling takes only 1 iteration compared to gradient descent without scaling, which takes 32.\
For both, the error in function value is less than precision of float.
Error in minimizer is of the order 1e-10 for GD without scaling

## 4)

In [11]:
X2 = np.array([8,8])
tol = 1e-9

try:
  find_minimizer_newton(X2,tol,1)
except Exception as ex:
  print(ex)  

x41,fx41,k41 = find_minimizer_newton(X2,tol,0.1)

Singular matrix
Iteration:191, Minimizer:[-1.3771743e-09 -1.3771743e-09], min value:4.0, gradient norm:9.738092830038097e-10


<ipython-input-2-8c8f351a8320>:17: RuntimeWarning: overflow encountered in power
  return np.array([[4/np.power(x1**2+4,1.5),0],[0,4/np.power(x2**2+4,1.5)]])


In [12]:
x42, fx42, k42 = find_minimizer_newton(X2,tol,1,backtrack=True)

Iteration:13, Minimizer:[2.83764947e-12 2.83764947e-12], min value:4.0, gradient norm:2.0065211811812668e-12


In [13]:
print(f'Newtons method with constant step length 1 : Minimizer {x41}, Min function value :{fx41} , Iterations:{k41}.\nNewtons method with backtracking line search: Minimizer {x42}, Min function value :{fx42} , Iterations:{k42}.')

Newtons method with constant step length 1 : Minimizer [-1.3771743e-09 -1.3771743e-09], Min function value :4.0 , Iterations:191.
Newtons method with backtracking line search: Minimizer [2.83764947e-12 2.83764947e-12], Min function value :4.0 , Iterations:13.


Newtons method with constant step length diverged for step length = 1 (leading to overflow).
If we reduce step size to 0.1, then it converges with 191 iterations.
While the method with backtracking search and starting step length=1 converged in 32 iterations. This is because the step lengths were reduced to an appropriate value before starting each iteration.\
For both, the error in function value is less than precision of float.\

Error in minimizers are of orders 1e-9 and 1e-12 respectively.

## 5)

In [14]:
x51, fx51, k51 = find_minimizer_gdscaling(X2,tol,1,0.5,0.5)

Iteration:13, Minimizer:[2.83764947e-12 2.83764947e-12], min value:4.0, gradient norm:2.0065211811812668e-12


In [15]:
x52, fx52, k52 = find_minimizer_gd(X2,tol,1,0.5,0.5)

Iteration:39, Minimizer:[8.3177047e-10 8.3177047e-10], min value:4.0, gradient norm:5.881505395106815e-10


In [16]:
print(f'Gradient descent with scaling : Minimizer {x51}, Min function value :{fx51} , Iterations:{k51}.\nGradient descent without scaling: Minimizer {x52}, Min function value :{fx52} , Iterations:{k52}.')

Gradient descent with scaling : Minimizer [2.83764947e-12 2.83764947e-12], Min function value :4.0 , Iterations:13.
Gradient descent without scaling: Minimizer [8.3177047e-10 8.3177047e-10], Min function value :4.0 , Iterations:39.


Since the hessian matrix is diagonal, it happens to coincide with our chosen scaling matrix. Hence, we get same solution for newtons method and scaled gradient descent.\
Gradient descent without scaling took 3 times more iterations.\
For both, the error in function value is less than precision of the float.\

Error in minimizers are of  orders 1e-12 and 1e-10 respectively